In [16]:
%websocketconnect --password wpass

** WebSocket connected **
Password: 
WebREPL connected
>>> Ready.


In [10]:
from machine import Pin

prelay = Pin(12, Pin.OUT)
pled = Pin(15, Pin.OUT)
pbutton = Pin(0, Pin.IN)

phlwselCF1 = Pin(5, Pin.OUT)  # 0 current, 1 voltage
phlwCF1 = Pin(13, Pin.IN)
phlwCF = Pin(14, Pin.IN)      # the power value


In [10]:
pled.value(0)

In [140]:
prelay.value(1)

In [24]:
phlwselCF1.value(0)

In [68]:
import time, array
ptimes = array.array("I", range(0,12)) # [i, tprev, dt, dt, ..., dt]
ptimes[1] = time.ticks_ms()
ptimes[0] = 1  # avoid zero elements for recpower
def pinirq(p):
    t1 = time.ticks_ms()
    ptimes[0] += 1
    ptimes[(ptimes[0]%(len(ptimes)-2))+2] = t1 - ptimes[1]
    ptimes[1] = t1

phlwCF.irq(trigger=Pin.IRQ_RISING | Pin.IRQ_FALLING, handler=pinirq)


In [18]:
import ubinascii; O6 = ubinascii.a2b_base64

In [2]:
%sendtofile --binary test.py
print(ptimes)
print(ptimes)
print(ptimes)
print(ptimes)
print(ptimes)print(ptimes)
print(ptimes)print(ptimes)
int(ptimes)print(ptimes)
print(ptimes)print(ptimes)
print(ptimes)print(ptimes)
print(ptimes)print(ptimes)

Sent 215 bytes in 8 chunks to test.py.


In [74]:
prelay.value(1)

In [7]:
%sendtofile test.py

# calculate the vital statistics
V_REF = 2.43
F_OSC = 3579000
HLW8012_CURRENT_R = 0.001 # 1milliOhm copper-manganese resistor in series with the main line
HLW8012_VOLTAGE_R_DOWN = 1000 # 1kOhm resistor in the voltage divider that feeds the V2P
HLW8012_VOLTAGE_R_UP = 5 * 470000 # 5 470kOhm resistors in the voltage divider that feeds the V2P pin
current_resistor = HLW8012_CURRENT_R
voltage_resistor = (HLW8012_VOLTAGE_R_UP + HLW8012_VOLTAGE_R_DOWN) / HLW8012_VOLTAGE_R_DOWN
current_multiplier = (1000.0 * 512 * V_REF / current_resistor / 24.0 / F_OSC)
voltage_multiplier = (1000.0 * 512 * V_REF * voltage_resistor / 2.0 / F_OSC)
power_multiplier = (1000.0 * 128 * V_REF * V_REF * voltage_resistor / current_resistor / 48.0 / F_OSC)
print(power_multiplier)
    

...

*** Sending Ctrl-C

[Timed out waiting for recognizable response]


In [112]:
def ytimes():
    i = ptimes[0]
    while True:
        while i < ptimes[0]:
            i += 1
            yield ptimes[(i%(len(ptimes)-2))+2]
        time.sleep_ms(10)

# I think the rolling max value is the best as avoids the noise on lowe power 
# and goes out of scope on high power (lots of low values)
def recpower():
    ctime = time.ticks_ms() - ptimes[1]
    maxtime = max(ptimes[(i%(len(ptimes)-2))+2]  for i in range(ptimes[0], max(0,ptimes[0]-6),-1))
    return max(ctime, maxtime)
    

In [144]:
print(power_multiplier/(recpower()*2))

12.3432


In [143]:
help("modules")

__main__          http_client_ssl   sys               urandom
_boot             http_server       time              ure
_onewire          http_server_ssl   uasyncio/__init__ urequests
_webrepl          inisetup          uasyncio/core     urllib/urequest
apa102            json              ubinascii         uselect
array             lwip              ucollections      usocket
btree             machine           uctypes           ussl
builtins          math              uerrno            ustruct
dht               micropython       uhashlib          utime
ds18x20           neopixel          uheapq            utimeq
errno             network           uio               uzlib
esp               ntptime           ujson             webrepl
example_pub_button                  onewire           umqtt/robust      webrepl_setup
example_sub_led   os                umqtt/simple      websocket
flashbdev         port_diag         uos               websocket_helper
framebuf          select            u

In [146]:
import machine
print(machine.unique_id())

b'Yo;\x00'


In [ ]:
####################
# the attempt to make a working goodnight lamp
####################

In [17]:
%sendtofile --binary main.py

from machine import Pin

prelay = Pin(12, Pin.OUT)
pled = Pin(15, Pin.OUT)
phlwCF = Pin(14, Pin.IN)      # the power value
prelay.value(1)
power_multiplier = 10343.6

# find the power times
import time, array
ptimes = array.array("I", range(0,12)) # [i, tprev, dt, dt, ..., dt]
ptimes[1] = time.ticks_ms()
ptimes[0] = 1  # avoid zero elements for recpower
def pinirq(p):
    t1 = time.ticks_ms()
    ptimes[0] += 1
    ptimes[(ptimes[0]%(len(ptimes)-2))+2] = t1 - ptimes[1]
    ptimes[1] = t1
phlwCF.irq(trigger=Pin.IRQ_RISING | Pin.IRQ_FALLING, handler=pinirq)

def recpower():
    ctime = time.ticks_ms() - ptimes[1]
    maxtime = max(ptimes[(i%(len(ptimes)-2))+2]  for i in range(ptimes[0], max(0,ptimes[0]-6),-1))
    return power_multiplier/(max(ctime, maxtime)*2)


import network, ubinascii
staif = network.WLAN(network.STA_IF)
staif.active(True)
doesssid = max((x  for x in staif.scan()  if x[0][:4] == b'DoES'), key=lambda X:X[3])[0]
staif.connect(doesssid, b'decafbad00')
while not staif.isconnected():
    pass
print(staif.ifconfig())
clientid = 'MicroPython-%s'%ubinascii.hexlify(network.WLAN().config('mac')[-3:]).decode()
print(clientid)

from umqtt.robust import MQTTClient
mqttbroker = "192.168.0.159"   # mqtt.local
client = MQTTClient(clientid, mqttbroker)

while True:
    pw = recpower()
    pled.value(0 if pw<5 else 1)
    print("pow", pw)
    client.connect()
    client.publish((clientid+"/ip").encode(), staif.ifconfig()[0].encode())
    client.publish((clientid+"/pow").encode(), str(pw).encode())
    client.publish(b"ESPURNA_A7A528/relay/0/set", (b"0" if pw<5 else b"1"))
    client.disconnect()
    time.sleep(5)

Sent 1641 bytes in 55 chunks to main.py.


In [8]:
import os
print(os.listdir())


['boot.py', 'webrepl_cfg.py', 'test.py']


In [18]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] 
MicroPython v1.9.3-8-g63826ac5c on 2017-11-01; ESP module with ESP8266
Type "help()" for more information.
>>> 
Closing websocket <websocket._core.WebSocket object at 0x7f6440591ef0>


In [ ]:
c.publish(b"ESPURNA_A7A528/relay/0/set", b"1")
